In [ ]:
Bulk Download | Reconciliation of data to Sync with Dail deals

In [26]:
import os
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Path to the folder with Excel files
inbound_folder_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\InboundDataMatch'

# Path to the MCAP31122023.xlsx file
mcap_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\BSE NSE FindingScripNumber\MCAP31122023.xlsx'

# Read the MCAP file to get the mapping between 'Company Name' and 'Symbol'
print("Reading MCAP file...")
mcap_df = pd.read_excel(mcap_file_path)

mcap_df['Company Name'] = mcap_df['Company Name'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()) if pd.notnull(x) and "Bank of" not in x else x)
mcap_df['Company Name'] = mcap_df['Company Name'].str.replace(r'\s*(ltd|LTD|Ltd)\.?$', '', regex=True, case=False)

# print(mcap_df['Company Name'])
company_name_to_symbol = dict(zip(mcap_df['Company Name'], mcap_df['Symbol']))

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through each file in the inbound folder
for file_name in os.listdir(inbound_folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(inbound_folder_path, file_name)
        
        # Read each Excel file into a DataFrame
        print(f"\nReading Excel file: {file_path}")
        df = pd.read_excel(file_path, header=1)  # Assuming header starts from the second row
        df.rename(columns={'Avg. Price': 'Average Price'}, inplace=True)

        df['Stock Name'] = df['Stock'].apply(lambda x: ' '.join(x.split()[:2]) if pd.notnull(x) and "Bank of" not in x else x)
        df['Stock Name'] = df['Stock Name'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x) if pd.notnull(x) else x).str.lower()


        # Check if the 'Stock Name' contains any 'Company Name' in the MCAP file
        print(f"Checking matches for file: {file_name}")
        df['Code'] = df['Stock Name'].apply(lambda stock_name: company_name_to_symbol[max(company_name_to_symbol.keys(), key=lambda company_name: fuzz.ratio(stock_name.lower(), company_name.lower()))] if pd.notnull(stock_name) else None)
        
        # print(df['Code'] )
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
print("\nCombining all DataFrames...")
combined_df = pd.concat(dfs, ignore_index=True)

# Define the output path and file name
output_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\InboundDataMatch\output'
output_file_name = f"{pd.Timestamp.now().strftime('%Y%m%d')}-DailyDeals-Trendlyne_Bulk.xlsx"
output_file_path = os.path.join(output_path, output_file_name)

# # Save the combined DataFrame to Excel with specific sheet name and column names
# print(f"\nSaving DataFrame to: {output_file_path}")
# with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
#     combined_df.to_excel(writer, index=False, sheet_name='Bulk Block Deals', startrow=1, header=False)
#     # Get the xlsxwriter workbook and worksheet objects
#     workbook = writer.book
#     worksheet = writer.sheets['Bulk Block Deals']
    
#     # Add the header to the worksheet
#     header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top'})
#     for col_num, value in enumerate(combined_df.columns.values):
#         worksheet.write(0, col_num + 1, value, header_format)

#     # Get the dimensions of the DataFrame
#     num_rows, num_cols = combined_df.shape
    
#     # Create a table (DataFrame) object
#     worksheet.add_table(0, 0, num_rows, num_cols, {'columns': [{'header': col} for col in combined_df.columns], 'name': 'BulkBlockDealsTable'})

#     # Add 'Code' as the 4th column (column D)
#     code_column = combined_df['Code']  # Replace 'Code' with the actual column name
#     worksheet.write_column(1, 3, code_column)  # Updated index to 3 for column D
# Save the combined DataFrame to Excel with specific sheet name and column names
print(f"\nSaving DataFrame to: {output_file_path}")
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    combined_df.to_excel(writer, index=False, sheet_name='Bulk Block Deals', startrow=1, header=False)
    # Get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Bulk Block Deals']
    
    # Add the header to the worksheet
    header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top'})
    for col_num, value in enumerate(combined_df.columns.values):
        worksheet.write(0, col_num + 1, value, header_format)

    # Get the dimensions of the DataFrame
    num_rows, num_cols = combined_df.shape
    
    # Create a table (DataFrame) object
    worksheet.add_table(0, 0, num_rows, num_cols, {'columns': [{'header': col} for col in combined_df.columns], 'name': 'BulkBlockDealsTable'})






Reading MCAP file...

Reading Excel file: C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\InboundDataMatch\20240129-DailyDeals-Trendlyne.xlsx
Checking matches for file: 20240129-DailyDeals-Trendlyne.xlsx

Combining all DataFrames...

Saving DataFrame to: C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\InboundDataMatch\output\20240130-DailyDeals-Trendlyne_Bulk.xlsx
